## Important packages

In [1]:
from __future__ import print_function
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt


In [2]:
cudnn.benchmark = True
manualSeed=random.randint(1, 10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  6512


## Downloading CIFAR10 Images

In [3]:
dataset=dset.CIFAR10(root='./data',download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))]))

dataloader=torch.utils.data.DataLoader(dataset,batch_size=64,shuffle=True,num_workers=2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")                        # availability of cuda devices


Files already downloaded and verified


## Hyperparameters for the DCGAN and Initialization of weights

In [4]:
from types import new_class

ngpu=1                                   # number of gpu
nz=100                                   # size of z latent vector (i.e. size of generator input)
ngf=32                                   # size of feature maps in generator
ndf=32                                   # size of feature maps in discriminator
nc=3

#initialization of weights on generator and discriminator
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

## DCGAN -Genarator network

In [5]:
# generator
class Generator(nn.Module):
  def __init__(self,ngpu,nc=3,nz=100,ngf=32):
    super(Generator,self).__init__()
    self.ngpu=ngpu
    self.main=nn.Sequential(
        nn.ConvTranspose2d(nz,ngf*8,4,1,0,bias=False),
        nn.BatchNorm2d(ngf*8),
        nn.ReLU(True),

        nn.ConvTranspose2d(ngf*8,ngf*4,4,2,1,bias=False),
        nn.BatchNorm2d(ngf*4),
        nn.ReLU(True),

        nn.ConvTranspose2d(ngf*4,ngf*2,4,2,1,bias=False),
        nn.BatchNorm2d(ngf*2),
        nn.ReLU(True),

        nn.ConvTranspose2d(ngf*2,ngf,4,2,1,bias=False),
        nn.BatchNorm2d(ngf),
        nn.ReLU(True),



        nn.ConvTranspose2d(ngf,nc,kernel_size=3,stride=1,padding=1,bias=False),
        nn.Tanh()
    )
  def forward(self,input):
    if input.is_cuda and self.ngpu>1:
      output=nn.parallel.data_parallel(self.main,input,range(self.ngpu))
    else:
      output=self.main(input)
    return output

netG=Generator(ngpu,nc,nz,ngf).to(device)
netG.apply(weights_init)
print(netG)




Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(32, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


## DCGAN Discriminator

In [6]:
#Discriminator
class Discriminator(nn.Module):
  def __init__(self,ngpu,nc=3,ndf=32):
    super(Discriminator,self).__init__()
    self.ngpu=ngpu
    self.main=nn.Sequential(
        # input layer nc X 32 x 32
        nn.Conv2d(nc,ndf,4,2,1,bias=False),
        nn.LeakyReLU(0.2,inplace=True),
        #state size(ndf x 16 x 16)

        nn.Conv2d(ndf,ndf*2,4,2,1,bias=False),
        nn.BatchNorm2d(ndf*2),
        nn.LeakyReLU(0.2,inplace=True),
        #state size(ndf*2 x 8 x 8)

        nn.Conv2d(ndf*2,ndf*4,4,2,1,bias=False),
        nn.BatchNorm2d(ndf*4),
        nn.LeakyReLU(0.2,inplace=True),
        #state size(ndf*4 x 4 x 4)

        nn.Conv2d(ndf*4,1,4,1,0,bias=False),
        nn.Sigmoid()
    )
  def forward(self,input):
    if input.is_cuda and self.ngpu>1:
      output=nn.parallel.data_parallel(self.main,input,range(self.ngpu))
    else:
      output=self.main(input)
    return output.view(-1,1).squeeze(1)
netD=Discriminator(ngpu).to(device)
netD.apply(weights_init)
print(netD)


Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(128, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (9): Sigmoid()
  )
)


## Training of the DCGAN

In [ ]:
criterion=nn.BCELoss()

#optimizer
optimizerD=optim.Adam(netD.parameters(),lr=0.0002,betas=(0.5,0.999))
optimizerG=optim.Adam(netG.parameters(),lr=0.0002,betas=(0.5,0.999))

fixed_noise=torch.randn(64,nz,1,1,device=device)
real_label=1
fake_label=0
niter=400
G_losses=[]
D_losses=[]

for epoch in range(niter):
  for i,data in enumerate(dataloader,0):
    # update the network: maximize log(D(x)+log(1-D(G(z))))

    #train with real data
    netD.zero_grad()
    real_cpu=data[0].to(device)
    batch_size=real_cpu.size(0)
    label=torch.full((batch_size,),real_label,dtype=torch.float,device=device)

    output=netD(real_cpu)
    #calculate loss on all-real batch
    errD_real=criterion(output,label)
    errD_real.backward()
    D_x=output.mean().item()
    #train fake
    noise=torch.randn(batch_size,nz,1,1,device=device)
    fake=netG(noise)
    label.fill_(fake_label)
    output=netD(fake.detach())
    errD_fake=criterion(output,label)
    errD_fake.backward()
    D_G_z1=output.mean().item()
    errD=errD_real+errD_fake
    optimizerD.step()

    # update G network: maximize(D(G(z)))
    netG.zero_grad()
    label.fill_(real_label)
    output=netD(fake)
    errG=criterion(output,label)
    errG.backward()
    D_G_z2=output.mean().item()
    optimizerG.step()

    G_losses.append(errG.item())
    D_losses.append(errD.item())
    if i%100==0:
      print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
          % (epoch, niter, i, len(dataloader),
             errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

[0/400][0/782] Loss_D: 1.5242 Loss_G: 1.1275 D(x): 0.4773 D(G(z)): 0.5118 / 0.3403
[0/400][100/782] Loss_D: 0.5347 Loss_G: 2.9798 D(x): 0.7667 D(G(z)): 0.2056 / 0.0558
[0/400][200/782] Loss_D: 0.9656 Loss_G: 1.7386 D(x): 0.6262 D(G(z)): 0.3546 / 0.1895
[0/400][300/782] Loss_D: 1.0962 Loss_G: 1.7830 D(x): 0.6355 D(G(z)): 0.4269 / 0.1817
[0/400][400/782] Loss_D: 0.9650 Loss_G: 1.9349 D(x): 0.6124 D(G(z)): 0.3133 / 0.1707
[0/400][500/782] Loss_D: 1.0913 Loss_G: 1.7556 D(x): 0.6795 D(G(z)): 0.4736 / 0.1898
[0/400][600/782] Loss_D: 0.9884 Loss_G: 1.8991 D(x): 0.5434 D(G(z)): 0.2642 / 0.1601
[0/400][700/782] Loss_D: 0.9558 Loss_G: 2.0387 D(x): 0.5589 D(G(z)): 0.2357 / 0.1442
[1/400][0/782] Loss_D: 0.9042 Loss_G: 1.5441 D(x): 0.6614 D(G(z)): 0.3527 / 0.2413
[1/400][100/782] Loss_D: 0.8802 Loss_G: 1.9049 D(x): 0.6231 D(G(z)): 0.2841 / 0.1690
[1/400][200/782] Loss_D: 0.8996 Loss_G: 2.3987 D(x): 0.7031 D(G(z)): 0.3734 / 0.1191
[1/400][300/782] Loss_D: 0.8036 Loss_G: 1.7344 D(x): 0.6401 D(G(z)): 

## Comparing the  DCGAN Generator with real images

In [ ]:
netG.eval()
netD.eval()
noise=torch.randn(8,nz,1,1,device=device)
with torch.no_grad():
  fake=netG(noise).detach().cpu()
plt.figure(figsize=(15,5))
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(vutils.make_grid(fake,padding=2,normalize=True,nrow=4).cpu(),(1,2,0)))
plt.savefig('fake_images.png')
plt.show()

## Latent space interpolation

In [ ]:
#latent space interpolation
z1=torch.randn(1,nz,1,1,device=device)
z2=torch.randn(1,nz,1,1,device=device)
num_steps=6
interpolates=[]
for i in range(num_steps):
  alpha=i/float(num_steps)
  interpolates.append((1-alpha)*z1+alpha*z2)
interpolates=torch.cat(interpolates,dim=0)
with torch.no_grad():
  fake=netG(interpolates).detach().cpu()
  plt.figure(figsize=(15,10))
  plt.axis("off")
  plt.title("Fake Images")
  plt.imshow(np.transpose(vutils.make_grid(fake,padding=2,normalize=True,nrow=10).cpu(),(1,2,0)))
  plt.savefig('interpolates.png')
  plt.show()
